In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer

dev = qml.device("default.qubit", wires = 4)
#  define device as 4 qubits

In [2]:
def layer(W):

    qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires = 0)
    qml.Rot(W[1, 0], W[1, 1], W[1, 2], wires = 0)
    qml.Rot(W[2, 0], W[2, 1], W[2, 2], wires = 0)
    qml.Rot(W[3, 0], W[3, 1], W[3, 2], wires = 0)

    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[3, 0])

Step1: Applying rotation
=> data: W[angle1, angle2, angle3] ...

Step2: Applying CNOT(entanglement)

In [3]:
def state_init(x):
    qml.BasisState(x, wires = [0, 1, 2, 3])

#  circuit defined as dev(linking), "autograd" => enabling auto tracking for gradient while training 
@qml.qnode(dev, interface= "autograd")
def circuit(weights, x):

    state_init(x)

    for W in weights:
        layer(W)

    return qml.expval(qml.PauliZ(0))

state_init(x):
=> classical data -> quantumstate

circuit(weights, x):
=> Set to quantumstate. Applying encoding(=rotation, CNOT) for varable weights(angle data in array)

return measurement on z axis

In [4]:
def variational_classifier(weights, bias, x):
    
    return circuit(weights, x) + bias

variational_classifier(weights, bias, x):
=> return prediciton value(y^) 

In [5]:
def square_loss(labels, predictions):
    loss = 0

    # for variables in zip(x, y) => x, y data를 variables에 mapping 해준다. 
    for l,p in zip(labels, predictions):
        loss = loss + (l-p)**2

    # len: return number of items
    loss = loss / len(labels)
    return loss


def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        # 1e-5 : 오차 감안 값(not exactly 0)
        if abs(1-p) < 1e-5:
            loss = loss + 1

    loss = loss / len(labels)

    return loss


def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]

    return square_loss(Y, predictions)


square_loss(labels, predictions):
=> Applying cost

COST: (1/N) * sigma_i(y-y_i)^2


accuracy(labels, predcitions):
=> calculating accuracy

returns how many inputs classifier predicted correctly(value of counter)


cost(weights, bias, X, Y):
=> calculating final cost value
    1) calculating predictions(y^) by using predicitons = variational_classifier(weights, bias, x) for x in X
    2) calculating square loss by using square_loss(Y, predictions(=y^))

In [6]:
data = np.loadtxt("/home/iridescent923/home/Pennylane_tutorial/parity.txt")

#  data[:, :-1 ] => 행*열: 마지막 열(row) cut out
X = np.array(data[:, :-1], requires_grad = False)

# Y = data[:, -1] => 마지막 열(col)만 포함 => 가로(row)로 저장 
Y = np.array(data[:, -1], requires_grad = False)
Y = Y * 2 - np.ones(len(Y))

for i in range(5):
    print("X = {}, Y = {: d}". format(X[i], int(Y[i])))


print(". . .")

X = [0. 0. 0. 0.], Y = -1
X = [0. 0. 0. 1.], Y =  1
X = [0. 0. 1. 0.], Y =  1
X = [0. 0. 1. 1.], Y = -1
X = [0. 1. 0. 0.], Y =  1
. . .


0 0 0 0 0
0 0 0 1 1
0 0 1 0 1
0 0 1 1 0

X = [:, :-1] and Y = [:, -1]

X = 마지막 col뺴고 저장; execpt last column 
Y = 마지막 col만 저장(= label)

In [7]:
#  generating random number
np.random.seed(0)

num_qubits = 4
num_layers = 2
#  requires_grad = True => track the gradient on training
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad = True)
bias_init = np.array(0.0, requires_grad = True)

print(weights_init)
print('\n')
print(bias_init)

[[[ 0.01764052  0.00400157  0.00978738]
  [ 0.02240893  0.01867558 -0.00977278]
  [ 0.00950088 -0.00151357 -0.00103219]
  [ 0.00410599  0.00144044  0.01454274]]

 [[ 0.00761038  0.00121675  0.00443863]
  [ 0.00333674  0.01494079 -0.00205158]
  [ 0.00313068 -0.00854096 -0.0255299 ]
  [ 0.00653619  0.00864436 -0.00742165]]]


0.0


weights_init = 0.01 * np.random.randn(2, 4, 3, requires_grad = True)

OUT:
[[[ 0.01764052  0.00400157  0.00978738]
  [ 0.02240893  0.01867558 -0.00977278]
  [ 0.00950088 -0.00151357 -0.00103219]
  [ 0.00410599  0.00144044  0.01454274]]

 [[ 0.00761038  0.00121675  0.00443863]
  [ 0.00333674  0.01494079 -0.00205158]
  [ 0.00313068 -0.00854096 -0.0255299 ]
  [ 0.00653619  0.00864436 -0.00742165]]]


=> 
random.randn => 난수 생성
array set = 2, row = 4, col = 3

In [8]:
opt = NesterovMomentumOptimizer(0.5)
batch_size = 5

weights = weights_init
bias = bias_init

for i in range(25):
    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            i + 1, cost(weights, bias, X, Y), acc)
    )

Iter:     1 | Cost: 3.4223141 | Accuracy: 0.0000000 
Iter:     2 | Cost: 1.3291641 | Accuracy: 0.5000000 
Iter:     3 | Cost: 1.8847136 | Accuracy: 0.5000000 
Iter:     4 | Cost: 1.9897607 | Accuracy: 0.5000000 
Iter:     5 | Cost: 1.2847100 | Accuracy: 0.0000000 
Iter:     6 | Cost: 2.6672535 | Accuracy: 1.0000000 
Iter:     7 | Cost: 1.0336971 | Accuracy: 0.5000000 
Iter:     8 | Cost: 1.8244553 | Accuracy: 0.5000000 
Iter:     9 | Cost: 1.3488304 | Accuracy: 1.0000000 
Iter:    10 | Cost: 1.2036406 | Accuracy: 0.0000000 
Iter:    11 | Cost: 1.9276973 | Accuracy: 0.5000000 
Iter:    12 | Cost: 1.6115580 | Accuracy: 0.5000000 
Iter:    13 | Cost: 2.3361288 | Accuracy: 0.0000000 
Iter:    14 | Cost: 1.9648592 | Accuracy: 0.5000000 
Iter:    15 | Cost: 1.9269579 | Accuracy: 0.5000000 
Iter:    16 | Cost: 2.1141187 | Accuracy: 0.5000000 
Iter:    17 | Cost: 1.0043716 | Accuracy: 0.0000000 
Iter:    18 | Cost: 1.5411777 | Accuracy: 0.5000000 
Iter:    19 | Cost: 1.9084307 | Accuracy: 0.50

Nesterov momentum optimizer 
: is used to find the optimal parameters for the variational quantum classifier

weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)
=> opt.step()실행하면: updated parameters, the bias, the first moment, and the second moment

first moment =  running average of the gradients
second moment = squared gradients
=> why?: to prevent the local minimum 

** batch = number of the data points  

=================================